In [1]:
# 네이버 영화 리뷰 크롤링

### Code


from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd

url = 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.naver?code=154282&type=after&onlyActualPointYn=N&onlySpoilerPointYn=N&order=sympathyScore&page=%7b%7d'
html = urlopen(url)
soup = BeautifulSoup(html,'html.parser')


In [2]:
%%time
rating_list = []
review_list = []

for page in range(1,6000):
    try:
        url = f'https://movie.naver.com/movie/bi/mi/pointWriteFormList.naver?code=154282&type=after&onlyActualPointYn=N&onlySpoilerPointYn=N&order=sympathyScore&page={page}'
        html = urlopen(url)
        soup = BeautifulSoup(html,'html.parser')
        for i in range(10):
            rating = soup.select('div > em')
            rating = rating[i].get_text()
            rating_list.append(rating)
            
            review = soup.find('span',{'id':f'_filtered_ment_{i}'})            
            review = review.get_text().strip()
            review_list.append(review)
    except:
        pass

print(len(rating_list))
print(rating_list)
print(len(review_list))
print(review_list)
# 더 배트맨

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [3]:
df = pd.DataFrame({"ratings": rating_list, "reviews": review_list})
df = df.drop_duplicates()
df

,ratings,reviews
0,10,원래 dc가 detective comics의 준말임. 이거에 가장 부합하는 영화인 ...
1,10,MCU나 DCEU 같은 분위기를 기대하는 사람이면 그냥 가지마라. 그런 영화 아니니...
2,10,전작 배트맨 영화들과 다른 장르 느낌 화려함 느와르 한편 본거 같음 난 다크나이트 ...
3,10,지루..할 수가 있나..?또 보고 싶은데
4,10,로버트 패틴슨 뜬다뜬다
...,...,...
5890,6,나는 복수다. = 어벤져스에 끼고싶다.
5891,2,잡아 쳐 죽이지 배트맨이 무슨 명탐정 코난이냐? 계속 수수께끼만 풀고있네
5892,1,진짜 너무너무너무너무너무나 지루하다세 시간에 걸친 졸음과의 대사투!!완전히 잠들지 ...
5893,1,"기생오래비 같은애가 배트맨이라니...개인적으론 불호....크리스챤베일, 벤애플랙같은..."


In [19]:
### Naver Developers - Open API
import urllib
# Open API 사용 요청
client_id = 'PG3kwbDlhgk80hJqt_Ob'
client_secret = 'fbMT9Q0JGb'


def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

def search_title(title):
    encText = urllib.parse.quote(title)
    url = 'https://openapi.naver.com/v1/search/movie.json?display=100&query=' + encText
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        d = json.loads(response_body.decode('utf-8'))
        if (len(d['items']) > 0):
            return d['items']
        else:
            return None

    else:
        print("Error Code:" + rescode)

In [20]:
m = search_title('스왈로 탐험대와 아마존 해적')

In [21]:
m

[{'title': '<b>스왈로 탐험대와 아마존 해적</b>',
  'link': 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=143399',
  'image': 'https://ssl.pstatic.net/imgmovie/mdi/mit110/1433/143399_P13_172551.jpg',
  'subtitle': 'Swallows and Amazons',
  'pubDate': '2016',
  'director': '필립파 로소프|',
  'actor': '켈리 맥도날드|앤드류 스캇|라프 스팰|',
  'userRating': '4.21'}]